In [20]:
import os
import sys
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Set up logging
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
import seaborn as sns
# Jupyter specific settings
%matplotlib inline
# 'seaborn' is not a valid style. Use a built-in style or a custom style sheet.
# You can list available styles with plt.style.available
# For now, let's use a default style that's similar to seaborn
plt.style.use('default')

from rich import print as rprint
from pprint import pprint


# ai
import marvin
import marvin.audio

from entries import *



In [16]:
call_script = """\

    Bull Halsey - medical student on gen neuro service
    call back
    831 060 5026


    Panel sent to Mayo
    Does mayo still have 

    neuro general service

    frank george marshal
    242317722
    serum

    want to do FGQ1B antibody, serum 
    currently undergoing plasmapheresis exchange

    1. do we have serum
    2. does mayo have serum

    """

rprint(call_script)

Bull Halsey - medical student on gen neuro service
    call back
    831 060 5026


    Panel sent to Mayo
    Does mayo still have 

    neuro general service

    frank george marshal
    242317722
    serum

    want to do FGQ1B antibody, serum 
    currently undergoing plasmapheresis exchange

    1. do we have serum
    2. does mayo have serum

In [4]:
raise Exception("dont run this block")  # record the user


user_audio = marvin.audio.record_phrase(after_phrase_silence=8)  # Record for 15 seconds
user_audio.save("call_audio_copy.wav")

[09/30/24 21:16:40] INFO     marvin.audio: Recording...                                                ]8;id=329856;file:///Users/christophersnyder/opt/anaconda3/envs/digi-path/lib/python3.11/site-packages/marvin/audio.py\audio.py]8;;\:]8;id=975575;file:///Users/christophersnyder/opt/anaconda3/envs/digi-path/lib/python3.11/site-packages/marvin/audio.py#153\153]8;;\

[09/30/24 21:17:45] INFO     marvin.audio: Recording finished.                                         ]8;id=438144;file:///Users/christophersnyder/opt/anaconda3/envs/digi-path/lib/python3.11/site-packages/marvin/audio.py\audio.py]8;;\:]8;id=292808;file:///Users/christophersnyder/opt/anaconda3/envs/digi-path/lib/python3.11/site-packages/marvin/audio.py#155\155]8;;\

In [22]:
call_audio = marvin.audio.Audio.from_path("call_audio.wav")

# call_audio.play()  # .play()
# transcribe the text
call_text = marvin.transcribe(call_audio)

2024-09-30 21:46:52,670 - INFO - HTTP Request: POST https://api.openai.com/v1/audio/transcriptions "HTTP/1.1 200 OK"


In [23]:
# Transcribed audio
# | echo: true
rprint(call_text)

Hey this is Bill Halsey, I'm a medical student on the General Neuroservice. I'm calling about a patient Frank 
George Marshall, MRN 242317722, date of birth March 1st 1977. This patient had a serum sample sent to Mayo three 
days ago and since then has had plasmapheresis and we'd like to test the FGQ1B antibody on serum. That's the test 
we want to do but again we're having trouble because we did plasmapheresis and now we can't detect the antibody so 
we're looking for an old sample. So we're wondering if either we in the lab at BGH have a sample or at Mayo they 
have a sample we could add on or send to Mayo to get the FGQ1B antibody. Thanks.

---

# | ![](../src/ai_logger/ss_service-literal.png){width=450px} ![t](<../src/ai_logger/Screenshot 2024-09-30 at 21.42.28.png>){width=450px}  ![](<../src/ai_logger/Screenshot 2024-09-30 at 21.46.07.png>){width=450px}


----

## Extract Patient Info

In [64]:
patinfo = marvin.extract(
    call_text, target=Patient, instructions="get the patient information"
)
rprint(call_text)
rprint(patinfo)

2024-09-30 22:19:16,803 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Hey this is Bill Halsey, I'm a medical student on the General Neuroservice. I'm calling about a patient Frank 
George Marshall, MRN 242317722, date of birth March 1st 1977. This patient had a serum sample sent to Mayo three 
days ago and since then has had plasmapheresis and we'd like to test the FGQ1B antibody on serum. That's the test 
we want to do but again we're having trouble because we did plasmapheresis and now we can't detect the antibody so 
we're looking for an old sample. So we're wondering if either we in the lab at BGH have a sample or at Mayo they 
have a sample we could add on or send to Mayo to get the FGQ1B antibody. Thanks.

[
    Patient(
        last_name='marshall',
        first_name='frank george',
        dob=datetime.datetime(1977, 3, 1, 0, 0, tzinfo=TzInfo(UTC)),
        mrn=242317722,
        sex=None,
        age=None
    )
]

## Extract Caller Info

In [65]:
callerinfo = marvin.extract(
    call_text, target=Caller, instructions="get the caller's information"
)
rprint(call_text)
rprint(callerinfo)

2024-09-30 22:20:47,955 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Hey this is Bill Halsey, I'm a medical student on the General Neuroservice. I'm calling about a patient Frank 
George Marshall, MRN 242317722, date of birth March 1st 1977. This patient had a serum sample sent to Mayo three 
days ago and since then has had plasmapheresis and we'd like to test the FGQ1B antibody on serum. That's the test 
we want to do but again we're having trouble because we did plasmapheresis and now we can't detect the antibody so 
we're looking for an old sample. So we're wondering if either we in the lab at BGH have a sample or at Mayo they 
have a sample we could add on or send to Mayo to get the FGQ1B antibody. Thanks.

[
    Caller(
        last_name='halsey',
        first_name='bill',
        callback_number='',
        clinical_service='Neurology',
        attending_doctor=None,
        caller_details='medical student on the general neuroservice'
    )
]

## DataBase Integrity Checks

In [42]:
marvin.classify(call_text, labels=["a real call", "an audio test"])

2024-09-30 22:00:34,250 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


'a real call'

In [45]:
marvin.classify("this is a ", labels=["a real call", "an audio test"])

2024-09-30 22:02:15,769 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


'an audio test'

## Summarizing Text

In [67]:
summary_text = marvin.cast(
    call_text, instructions="summarize briefly in 10 words or less"
)
rprint(call_text)
rprint(summary_text)

2024-09-30 22:22:23,956 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Hey this is Bill Halsey, I'm a medical student on the General Neuroservice. I'm calling about a patient Frank 
George Marshall, MRN 242317722, date of birth March 1st 1977. This patient had a serum sample sent to Mayo three 
days ago and since then has had plasmapheresis and we'd like to test the FGQ1B antibody on serum. That's the test 
we want to do but again we're having trouble because we did plasmapheresis and now we can't detect the antibody so 
we're looking for an old sample. So we're wondering if either we in the lab at BGH have a sample or at Mayo they 
have a sample we could add on or send to Mayo to get the FGQ1B antibody. Thanks.

Need old serum sample for FGQ1B antibody test.

## Finally, Extract all Call Data 


In [69]:
call_entry = marvin.cast(
    call_text,
    target=CallEntry,
    instructions="cast as much of the call text dialog as possible",
)
rprint(call_text)
rprint(call_entry)

2024-09-30 22:23:59,196 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Hey this is Bill Halsey, I'm a medical student on the General Neuroservice. I'm calling about a patient Frank 
George Marshall, MRN 242317722, date of birth March 1st 1977. This patient had a serum sample sent to Mayo three 
days ago and since then has had plasmapheresis and we'd like to test the FGQ1B antibody on serum. That's the test 
we want to do but again we're having trouble because we did plasmapheresis and now we can't detect the antibody so 
we're looking for an old sample. So we're wondering if either we in the lab at BGH have a sample or at Mayo they 
have a sample we could add on or send to Mayo to get the FGQ1B antibody. Thanks.

CallEntry(
    service='Chemistry',
    patient=Patient(
        last_name='marshall',
        first_name='frank george',
        dob=datetime.datetime(1977, 3, 1, 0, 0, tzinfo=TzInfo(UTC)),
        mrn=242317722,
        sex=None,
        age=None
    ),
    caller=Caller(
        last_name='halsey',
        first_name='bill',
        callback_number='',
        clinical_service='Neurology',
        attending_doctor=None,
        caller_details='medical student on the general neuroservice'
    ),
    laboratory_test='FGQ1B antibody',
    call_category='Lab Problem',
    call_details="This patient had a serum sample sent to Mayo three days ago and since then has had plasmapheresis
and we'd like to test the FGQ1B antibody on serum. We're having trouble because we did plasmapheresis and now we 
can't detect the antibody so we're looking for an old sample. We're wondering if either we in the lab at BGH have a
sample or at Mayo they have a sample we could add on or send to Mayo to get the FGQ1B antibody.",
    specimen_type='serum',
    resolution='New'
)

## Save Entry
### append to the json file

In [59]:
from matplotlib.font_manager import json_dump

call_log_path = "call_log.json"
json_entry = call_entry.model_dump_json(indent=4)
with open(call_log_path, "a") as f:
    f.write(json_entry)

In [63]:
# Show scrollable JSON call_log.json


with open("call_log.json", "r") as f:
    call_log_data = json.load(f)

rprint(call_log_data)

{
    'service': 'Chemistry',
    'patient': {
        'last_name': 'marshall',
        'first_name': 'frank george',
        'dob': '1977-03-01T00:00:00Z',
        'mrn': 242317722,
        'sex': None,
        'age': None
    },
    'caller': {
        'last_name': 'halsey',
        'first_name': 'bill',
        'callback_number': '',
        'clinical_service': 'Neurology',
        'attending_doctor': None,
        'caller_details': 'medical student on the general neuroservice'
    },
    'laboratory_test': 'FGQ1B antibody',
    'call_category': 'Lab Problem',
    'call_details': "This patient had a serum sample sent to Mayo three days ago and since then has had 
plasmapheresis and we'd like to test the FGQ1B antibody on serum. We're having trouble because we did 
plasmapheresis and now we can't detect the antibody so we're looking for an old sample. We're wondering if either 
we in the lab at BGH have a sample or at Mayo they have a sample we could add on or send to Mayo to get the FGQ1B 
antibody.",
    'specimen_type': 'serum',
    'resolution': 'New'
}

In [44]:
# cast the language to a more formal style


# generate AI speech
ai_audio = marvin.speak(ai_text)

# play the result
ai_audio.play()

2024-09-30 22:01:09,424 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-30 22:01:12,091 - INFO - HTTP Request: POST https://api.openai.com/v1/audio/speech "HTTP/1.1 200 OK"


In [71]:
call_entries = [call_entry, call_entry, call_entry]

rprint(call_entries)


@marvin.fn
def add_patient_ages(call_entries: list[CallEntry]):
    """add up the ages of all the patients in the call log so far"""


print("the ages are", add_patient_ages(call_entries))

[
    CallEntry(
        service='Chemistry',
        patient=Patient(
            last_name='marshall',
            first_name='frank george',
            dob=datetime.datetime(1977, 3, 1, 0, 0, tzinfo=TzInfo(UTC)),
            mrn=242317722,
            sex=None,
            age=None
        ),
        caller=Caller(
            last_name='halsey',
            first_name='bill',
            callback_number='',
            clinical_service='Neurology',
            attending_doctor=None,
            caller_details='medical student on the general neuroservice'
        ),
        laboratory_test='FGQ1B antibody',
        call_category='Lab Problem',
        call_details="This patient had a serum sample sent to Mayo three days ago and since then has had 
plasmapheresis and we'd like to test the FGQ1B antibody on serum. We're having trouble because we did 
plasmapheresis and now we can't detect the antibody so we're looking for an old sample. We're wondering if either 
we in the lab at BGH have a sample or at Mayo they have a sample we could add on or send to Mayo to get the FGQ1B 
antibody.",
        specimen_type='serum',
        resolution='New'
    ),
    CallEntry(
        service='Chemistry',
        patient=Patient(
            last_name='marshall',
            first_name='frank george',
            dob=datetime.datetime(1977, 3, 1, 0, 0, tzinfo=TzInfo(UTC)),
            mrn=242317722,
            sex=None,
            age=None
        ),
        caller=Caller(
            last_name='halsey',
            first_name='bill',
            callback_number='',
            clinical_service='Neurology',
            attending_doctor=None,
            caller_details='medical student on the general neuroservice'
        ),
        laboratory_test='FGQ1B antibody',
        call_category='Lab Problem',
        call_details="This patient had a serum sample sent to Mayo three days ago and since then has had 
plasmapheresis and we'd like to test the FGQ1B antibody on serum. We're having trouble because we did 
plasmapheresis and now we can't detect the antibody so we're looking for an old sample. We're wondering if either 
we in the lab at BGH have a sample or at Mayo they have a sample we could add on or send to Mayo to get the FGQ1B 
antibody.",
        specimen_type='serum',
        resolution='New'
    ),
    CallEntry(
        service='Chemistry',
        patient=Patient(
            last_name='marshall',
            first_name='frank george',
            dob=datetime.datetime(1977, 3, 1, 0, 0, tzinfo=TzInfo(UTC)),
            mrn=242317722,
            sex=None,
            age=None
        ),
        caller=Caller(
            last_name='halsey',
            first_name='bill',
            callback_number='',
            clinical_service='Neurology',
            attending_doctor=None,
            caller_details='medical student on the general neuroservice'
        ),
        laboratory_test='FGQ1B antibody',
        call_category='Lab Problem',
        call_details="This patient had a serum sample sent to Mayo three days ago and since then has had 
plasmapheresis and we'd like to test the FGQ1B antibody on serum. We're having trouble because we did 
plasmapheresis and now we can't detect the antibody so we're looking for an old sample. We're wondering if either 
we in the lab at BGH have a sample or at Mayo they have a sample we could add on or send to Mayo to get the FGQ1B 
antibody.",
        specimen_type='serum',
        resolution='New'
    )
]

2024-09-30 22:29:05,445 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


the ages are 138


3*47

In [75]:
marvin.extract(
    call_entries,
    target=list[str],
    instructions="get the last names of all the callers so far",
)

2024-09-30 22:31:50,991 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[['halsey'], ['halsey'], ['halsey']]

In [ ]:
# ai = marvin.Application(call_entries)